In [ ]:
import openai
import os
import requests
import json
from concurrent.futures import ThreadPoolExecutor

from pathlib import Path

# load_dotenv()

openai.api_key = ""

In [ ]:
source_text = Path("YOUR_TEXT").read_text()
print(source_text)

In [ ]:
def split_text(text, max_length=1024):
    chunks = []
    words = text.split()
    current_chunk = ""
    for word in words:
        if len(current_chunk) + len(word) < max_length:
            current_chunk += f" {word}"
        else:
            chunks.append(current_chunk.strip())
            current_chunk = f"{word}"
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [ ]:
def generate_summary(text):
    input_chunks = split_text(text)
    output_chunks = []
    for chunk in input_chunks:
        response = openai.ChatCompletion.create(
            model="YOUR_DEPLOYMENT",
            messages =[ 
                {"role": "system", "content": "This text is a chunked summary of a transcript. Create a coherent summary based on the text. Avoid repeating names and do not mention when participants join or leave the meeting"},
                {"role": "user", "content": chunk}
            ],
            temperature=0.5,
            max_tokens=1024,
        )
        summary = response['choices'][0]['message']['content']
        output_chunks.append(summary)

    concat_text= " ".join(output_chunks)
    print(concat_text)
    return concat_text

In [ ]:
def summary_of_summaries(summary_text):
    response = openai.ChatCompletion.create(
        model="YOUR_DEPLOYMENT",
        messages =[ 
            {"role": "system", "content": "This is a transcript from a class on Power BI. Extract and list the key points from the provided text. Group the key points by topic and display each topic group first with the topic in bold and then the summary of the topic below. Avoid repeating names and do not mention when participants join or leave the meeting"},
            {"role": "user", "content": summary_text}
        ],
        temperature=0.7,
        max_tokens=1500,
        
    )
    summary_of_summary = response['choices'][0]['message']['content']
    return summary_of_summary

In [ ]:
responses = summary_of_summaries(source_text)
print(responses)